# ReAct Agent with Multiple Tools

## Introduction

This project focuses on the implementation of a ReAct (Reasoning and Acting) agent inspired by the work of [Yao et al](https://arxiv.org/abs/2210.03629). The agent is designed to handle complex queries by effectively reasoning about which tool to use and when, leveraging a suite of three primary tools: search, compare, and analyze.

### Objectives:
1. **Search Tool**: Integration of SerpAPI to perform web searches, with results formatted for efficient use by the ReAct agent.
2. **Comparison Tool**: A custom tool enabling the agent to compare multiple items across specified categories.
3. **Analysis Tool**: Utilization of Gemini v1.5 model (from Google Vertex AI) to summarize and extract key insights from search results and comparisons.

This project bridges advanced reasoning techniques with practical applications, showcasing the potential of ReAct agents in solving real-world challenges.

In [ ]:
# Install required packages
!pip install langchain-google-vertexai google-search-results langchain
!pip install -U langchain-community

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-non

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
# Import necessary libraries
from langchain import LLMChain, PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import SerpAPIWrapper
from langchain.llms.base import LLM
from langchain_google_vertexai import ChatVertexAI

# Set API keys
import os
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

if not os.environ['SERPAPI_API_KEY']:
    print("SERPAPI_API_KEY is not set. Please configure the environment variable.")

# Initialize the language model
llm = ChatVertexAI(
    model="gemini-1.5-flash-002",
    temperature=0
)

## Implementing the search tool

In [ ]:
# Loading the search tool using SerpAPI
search_tool = SerpAPIWrapper()

search_instance = Tool(
    name="Search",
    description="A tool to search for information on the internet.",
    func=search_tool.run
)

## Creating a custom comparison tool

In [ ]:
# Custom tool for general comparison
comparison_prompt_template = """
Compare the following items in terms of the given category:
Items: {items}
Category: {category}
Provide a detailed comparison.

Comparison:
"""
comparison_prompt = PromptTemplate(template=comparison_prompt_template, input_variables=["items", "category"])
comparison_chain = LLMChain(llm=llm, prompt=comparison_prompt)

def compare_items(query: str) -> str:
    """Compare items based on the query."""
    all_input = query.split(":")

    # Create a prompt using the extracted items and category
    items_str = all_input[0]
    category = all_input[1]
    comparison_input = {"items": items_str, "category": category}

    # Run the comparison chain with the prompt input
    comparison_result = comparison_chain.run(comparison_input)

    # Return the formatted comparison result
    return f"{comparison_result}\n-----\nComparison completed. Proceed to analysis of the comparison."

compare_instance = Tool(
    name="Compare",
    description="A tool to compare multiple items based on a specific category.",
    func=compare_items
)

<ipython-input-4-c9d029d11426>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  comparison_chain = LLMChain(llm=llm, prompt=comparison_prompt)


## Implementing an analysis tool

In [ ]:
analysis_prompt_template = """
Analyze the following content to extract the key information and insights:
Content: {content}
Provide a concise summary.

Summary:
"""

analysis_prompt = PromptTemplate(template=analysis_prompt_template, input_variables=["content"])
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)

def analyze_results(results: str) -> str:
    """Analyze search and comparison results"""
    # If query is empty or doesn't request comparison, provide a sample query
    if not results:
        return "No data provided for analysis."

    # Run analysis
    analysis_input = {"content": results}
    analysis_summary = analysis_chain.run(analysis_input)

    return f"Analysis Summary:\n{analysis_summary}"

analyze_instance = Tool(
    name="Analyze",
    description="A tool to analyze and summarize content before presenting it to user.",
    func=analyze_results
)

## Final Step - Integrating tools with a ReAct agent

In [ ]:
# Integrate tools with ReAct agent
agent_tools = [search_instance, compare_instance, analyze_instance]
react_agent = initialize_agent(agent_tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


<ipython-input-6-ea6b46e23355>:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  react_agent = initialize_agent(agent_tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [ ]:
# Process query function
def process_query(query: str, max_steps: int = 100) -> str:
  # YOUR CODE HERE
  response = react_agent({"input": query, "max_steps": max_steps})
  return response


## Testing the Agent

In [ ]:
# Test your implementation
sample_query = "What are the top 3 smartphones in 2023, and how do they compare in terms of camera quality and battery life?"

result = process_query(sample_query)

<ipython-input-7-310341b11fb3>:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = react_agent({"input": query, "max_steps": max_steps})




> Entering new AgentExecutor chain...
Thought: I need to find the top 3 smartphones of 2023 first.  Then I need to compare their camera quality and battery life.

Action: Search
Action Input: "Top 3 smartphones 2023"

Observation: ['Best phone 2024 · 1. Samsung Galaxy S24 Ultra · 2. Xiaomi 14 Ultra · 3. Google Pixel 9 Pro XL · 4. Apple iPhone 16 Plus · 5. OnePlus Nord 4 · 6.', 'Top 20 phones of the year 2023 · Samsung Galaxy S23 Ultra. RANK: 1 · Samsung Galaxy A54. RANK: 2 · Samsung Galaxy S23. RANK: 3 · Samsung Galaxy A34.', 'The Best Phones · 1. Apple iPhone 16 Pro Max. View at Visible · 2. Google Pixel 9 Pro. View at Amazon · 3. Apple iPhone 16 Pro and iPhone 16 Pro ...', '1. iPhone 16 Pro Max: Best overall · 2. Galaxy S24 Ultra: Best Samsung phone · 3. Pixel 9 Pro: Smartest phone · 4. Pixel 8a: Best under $500 · 5.', 'Our pick for the best phone overall is the Samsung Galaxy S24 Ultra, thanks to its immersive and vibrant display, reliable battery life, and wealth of special ...',

<ipython-input-4-c9d029d11426>:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  comparison_result = comparison_chain.run(comparison_input)



Observation: ## Camera Quality & Battery Life Comparison: Samsung Galaxy S24 Ultra, Google Pixel 9 Pro, iPhone 16 Pro Max

This comparison is based on anticipated specifications and early reviews (as the S24 Ultra and iPhone 16 Pro Max are not yet fully released at the time of writing).  Actual performance may vary slightly.

**Camera Quality:**

* **Samsung Galaxy S24 Ultra:**  Expected to lead in overall megapixel count and zoom capabilities, boasting a potentially massive main sensor and a highly advanced periscope telephoto lens.  Samsung typically excels in detail capture and dynamic range, though sometimes at the cost of slightly less natural color reproduction than Google's Pixel line.  Expect excellent low-light performance, aided by advanced image processing.  Video recording capabilities are also anticipated to be top-tier.

* **Google Pixel 9 Pro:**  Google's strength lies in computational photography.  While megapixel counts might be lower than Samsung's, expect superior i

In [ ]:
print(result['output'])

The top 3 smartphones of 2023, according to various sources, are the Samsung Galaxy S24 Ultra, Google Pixel 9 Pro, and iPhone 16 Pro Max.  In terms of camera quality, the Samsung Galaxy S24 Ultra is expected to lead in megapixels and zoom, while the Google Pixel 9 Pro excels in computational photography for natural-looking images and AI features. The iPhone 16 Pro Max offers a balanced approach with strong performance and ease of use.  For battery life, all three offer excellent performance, with the Samsung Galaxy S24 Ultra and iPhone 16 Pro Max potentially lasting the longest, even under heavy use. The best choice depends on individual priorities: maximum detail and zoom (S24 Ultra), natural photos and AI (Pixel 9 Pro), or a balanced and user-friendly experience (iPhone 16 Pro Max).
